# Farmer example

Reference: Birge and Louveaux, "Introduction to Stochastic Programming", Chapter 1

The implementation directly follows the documentation of StochasticPrograms.jl: https://martinbiel.github.io/StochasticPrograms.jl/stable/manual/examples/

In [31]:
using StochasticPrograms
using HiGHS

In [32]:
using Random, Distributions

We will add some benchmark tools in order to better evaluate the efficiency of each approach.

In [33]:
using BenchmarkTools

### Cost Minimizing News Vendor Program
Using StochasticPrograms.jl's syntax, we describe the two-stage linear program.

In [124]:
procurement_c = 1.0 # procurement cost
holding_c = 1.0 # holding cost
stockout_c = 1.5*procurement_c # stockout cost

@stochastic_model name_model begin
    @stage 1 begin
        @parameters begin
            c = procurement_c
        end
        @decision(name_model, 0 <= x <= 10000)
        @objective(name_model, Min, x * c)
    end
    @stage 2 begin
        @parameters begin
            h = holding_c
            v = stockout_c
        end
        @uncertain ξ
        @recourse(name_model, x >= y >= 0)
        @recourse(name_model, ξ >= w >= 0)
        
        @objective(name_model, Min, h * y + v * w)
        @constraint(name_model, y >= x-ξ)
        @constraint(name_model, w >= ξ-x)
    end
end

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


In [88]:
procurement_c = [1.0, 100.0, 1000.0] # procurement cost
holding_c = 1.0 # holding cost
stockout_c = [[1.5*c, 5*c, 10*c] for c in procurement_c] # stock out cost

x = Vector{StochasticModel}()

for c in procurement_c 
    stockout_c = [1.5*c, 5*c, 10*c]
    for v in stockout_c 
        name = "newsvendor_c$(c)_v$(v)"

        @stochastic_model name begin
            @stage 1 begin
                @parameters begin
                    c = c
                end
                @decision(name, 0 <= x <= 10000)
                @objective(name, Min, x * c)
            end
            @stage 2 begin
                @parameters begin
                    h = holding_c
                    v = v
                end
                @uncertain ξ
                @recourse(name, x >= y >= 0)
                @recourse(name, ξ >= w >= 0)
                
                @objective(name, Min, h * y + v * w)
                @constraint(name, y >= x-ξ)
                @constraint(name, w >= ξ-x)
            end
        end
        push!(x, name)
    end
end

In [90]:
x[1].parameters

(StageParameters{NamedTuple{(:c,), Tuple{Float64}}}([:c], (c = 1.0,)), StageParameters{NamedTuple{(:h, :v), Tuple{Float64, Float64}}}([:h, :v], (h = 1.0, v = 1.5)))

Generate Scenarios

In [35]:
function generate_scenarios(distrib, max_val = 200)
    scenarios = [@scenario ξ = n probability = cdf(distrib, n) - cdf(distrib, n-1) for n in 1:max_val]
    return scenarios
end

generate_scenarios (generic function with 2 methods)

In [36]:
Random.seed!(42)

TaskLocalRNG()

In [37]:
Unif = Uniform(0, 200)
Exp1 = truncated(Exponential(100), 0, 200)
Exp2 = truncated(Exponential(1e6), 0, 200)

N1 = truncated(Normal(100, 10), 0, 200)
N2 = truncated(Normal(100, 50), 0, 200)
N3 = truncated(Normal(100, 100), 0, 200)

Trian = truncated(TriangularDist(0, 200, 100), 0, 200)  
distribs = [Unif, Exp1, Exp2, N1, N2, N3, Trian] 


7-element Vector{Distribution{Univariate, Continuous}}:
 Uniform{Float64}(a=0.0, b=200.0)
 Truncated(Exponential{Float64}(θ=100.0); lower=0.0, upper=200.0)
 Truncated(Exponential{Float64}(θ=1.0e6); lower=0.0, upper=200.0)
 Truncated(Normal{Float64}(μ=100.0, σ=10.0); lower=0.0, upper=200.0)
 Truncated(Normal{Float64}(μ=100.0, σ=50.0); lower=0.0, upper=200.0)
 Truncated(Normal{Float64}(μ=100.0, σ=100.0); lower=0.0, upper=200.0)
 Truncated(TriangularDist{Float64}(a=0.0, b=200.0, c=100.0); lower=0.0, upper=200.0)

In [38]:
tot_scenarios = [generate_scenarios(d) for d in distribs]

7-element Vector{Vector{Scenario{NamedTuple{(:ξ,), Tuple{Int64}}}}}:
 [Scenario with probability 0.005
  ξ: 1, Scenario with probability 0.005
  ξ: 2, Scenario with probability 0.004999999999999999
  ξ: 3, Scenario with probability 0.005000000000000001
  ξ: 4, Scenario with probability 0.005000000000000001
  ξ: 5, Scenario with probability 0.0049999999999999975
  ξ: 6, Scenario with probability 0.0050000000000000044
  ξ: 7, Scenario with probability 0.0049999999999999975
  ξ: 8, Scenario with probability 0.0049999999999999975
  ξ: 9, Scenario with probability 0.0050000000000000044
  ξ: 10  …  Scenario with probability 0.0050000000000000044
  ξ: 191, Scenario with probability 0.0050000000000000044
  ξ: 192, Scenario with probability 0.0050000000000000044
  ξ: 193, Scenario with probability 0.0050000000000000044
  ξ: 194, Scenario with probability 0.0050000000000000044
  ξ: 195, Scenario with probability 0.0050000000000000044
  ξ: 196, Scenario with probability 0.0050000000000000044
  ξ:

## Deterministic equivalent

We finally pass the complete model to our solver. By default, the deterministic equivalent problem is built.

In [44]:
tot_scenarios[1][2]

Scenario with probability 0.005
  ξ: 2

In [70]:
for i in 1:length(tot_scenarios)
    println("Distribution: ", distribs[i])
    println("Number of scenarios: ", length(tot_scenarios[i]))
    vals = [s.:probability for s in tot_scenarios[i]]
    println(sum(vals))
    println("hey", [s for s in vals[i]])
    # println("Total probability: ", sum([s.probability for s in vals[i]]))
    println()
end
# farmer = instantiate(newsv_cost_model, [ξ₁,ξ₂,ξ₃], optimizer = HiGHS.Optimizer)

Distribution: Uniform{Float64}(a=0.0, b=200.0)
Number of scenarios: 200


MethodError: MethodError: no method matching +(::Probability, ::Probability)

Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:578
  +(!Matched::Union{MathOptInterface.ScalarAffineFunction{T}, MathOptInterface.ScalarQuadraticFunction{T}}, ::T) where T
   @ MathOptInterface ~/.julia/packages/MathOptInterface/RuRWI/src/Utilities/functions.jl:1767
  +(!Matched::Union{AffineDecisionFunction{T}, QuadraticDecisionFunction{T}}, ::T) where T
   @ StochasticPrograms ~/.julia/packages/StochasticPrograms/pIScW/src/types/decisions/functions/operators.jl:251
  ...


We can check the model formulation.

In [125]:
# for model in x
    
#         println("Model\t", model)
model = x[2]
println("Model\t", name)
println("Parameters\t", name.parameters)
println(typeof(name))
println()
for i in tot_scenarios[7]
    if !(i isa Scenario)
        println("hey")
    end
end
# println("Scenarios\t", typeof(tot_scenarios[7][1]))
inst = instantiate(name_model, tot_scenarios[1], optimizer = HiGHS.Optimizer)

        # for i in length(tot_scenarios)
        #     println("Distribution: ", distribs[i])
        #     println("i", i)
        #     newsvendor = instantiate(model, tot_scenarios[i], optimizer = HiGHS.Optimizer)
            
        
    # end
# end

Model	Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)

Parameters	(StageParameters{NamedTuple{(:c,), Tuple{Float64}}}([:c], (c = 1.0,)), StageParameters{NamedTuple{(:h, :v), Tuple{Float64, Float64}}}([:h, :v], (h = 1.0, v = 1.5)))
StochasticModel{2, Tuple{StageParameters{NamedTuple{(:c,), Tuple{Float64}}}, StageParameters{NamedTuple{(:h, :v), Tuple{Float64, Float64}}}}}



ErrorException: The objective function `DecisionAffExpr{Float64}[x, 100 x, 1000 x]` is not supported by JuMP.

In [ ]:
println(farmer)

We can now run the optimization solver. We also print the first-stage decision as well as the optimal value.

`StochasticPrograms` also allows to compute the expected value of perfect information...

In [ ]:
println("EVPI: $(EVPI(farmer))")

... and the value of the stochastic solution.

In [ ]:
println("VSS: $(VSS(farmer))")

## L-shaped decomposition

Instead of the deterministic equivalent, we can use the L-shaped decomposition method.

### Multi-cuts

Let gather the scenario in one vector.

In [ ]:
ξ = [ξ₁, ξ₂, ξ₃]

We instantitate the model with the L-shaped method by specifying it as the chosen optimizer. By default, the multi-cut approach is selected.

In [ ]:
farmer_lshaped = instantiate(farmer_model, ξ, optimizer = LShaped.Optimizer)

The model is now explicitly a two-stage model.

In [ ]:
print(farmer_lshaped)

We have to specify the solver for each stage. This allows to choose more adapted algorithms

In [ ]:
set_optimizer_attribute(farmer_lshaped, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(farmer_lshaped, SubProblemOptimizer(), HiGHS.Optimizer)

We are now in position to solve the program. `StochasticPrograms` informs us on the number of iterations and generated cuts.

In [ ]:
optimize!(farmer_lshaped)

Let measure the performances.

In [ ]:
@benchmark optimize!(farmer_lshaped)

We want to compare it with the simple cut version, so we ask the program to aggregate the cuts.

In [ ]:
set_optimizer_attribute(farmer_lshaped, Aggregator(), Aggregate())

If we solve the problem, we now see that we need more iterations, while a few less cuts are generated. As expected, one cut is now produced at each iteration, except the last one, that established the convergence.

In [ ]:
optimize!(farmer_lshaped)

The benchmark also exhibits that the single cut technique is significantly slower.

In [ ]:
@benchmark optimize!(farmer_lshaped)